In [1]:
import streamlit as st
import pandas as pd
import numpy as np

In [2]:
horizontal_kernel = np.array([[-1, -1, -1],
                              [ 2,  2,  2],
                              [-1, -1, -1]])

vertical_kernel = np.array([[-1,  2, -1],
                            [-1,  2, -1],
                            [-1,  2, -1]])

def apply_convolution (matrix, kernel_type='horizontal'):
    if kernel_type == 'horizontal':
        kernel = np.array([[1, 1, 1], [0, 0, 0], [-1, -1, -1]])
    elif kernel_type == 'vertical':
        kernel = np.array([[1, 0, -1], [1, 0, -1], [1, 0, -1]])
    else:
        raise ValueError("El tipo de kernel debe ser 'horizontal' o 'vertical'")
    
    rows, cols = matrix.shape
    output = np.zeros((rows - 2, cols - 2))
    
    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            region = matrix[i-1:i+2, j-1:j+2]
            output[i-1, j-1] = np.sum(region * kernel)
    
    return output


In [3]:
def add_padding(matrix, padding_size):
    padded_matrix = np.pad(matrix, pad_width=padding_size, mode='constant', constant_values=0)
    return padded_matrix



In [4]:
def apply_stride_convolution(matrix, kernel_type='horizontal', stride=2):
    convolved_matrix = apply_convolution(matrix, kernel_type)
    result = convolved_matrix[::stride, ::stride]
    return result



In [5]:
def generate_feature_maps(matrix, n):
    feature_maps = []
    for i in range(n):
        if i % 2 == 0:
            feature_map = apply_convolution(matrix, 'horizontal')
        else:
            feature_map = apply_convolution(matrix, 'vertical')
        feature_maps.append(feature_map)
    return np.stack(feature_maps, axis=-1)



In [6]:
def max_pooling(matrix, stride=2):
    output_shape = (matrix.shape[0] // stride, matrix.shape[1] // stride)
    pooled_matrix = np.zeros(output_shape)
    
    for i in range(0, matrix.shape[0] - 1, stride):
        for j in range(0, matrix.shape[1] - 1, stride):
            pooled_matrix[i//stride, j//stride] = np.max(matrix[i:i+2, j:j+2])
    
    return pooled_matrix

In [9]:
data = pd.read_excel('C:/Users/totoy/Documents/UNIVERSIDAD/PYTHON/PARCIAL 2/PUNTO 1/pixeles.xlsx', header=None)
matrix = data.values  # Convertir a una matriz NumPy

In [10]:
conv_horizontal = apply_convolution(matrix, kernel_type='horizontal')

conv_vertical = apply_convolution(matrix, kernel_type='vertical')

print(conv_horizontal)
print(conv_vertical)


[[ 0.  0.  0.  0.]
 [ 1.  1.  0.  0.]
 [-1. -2. -1. -1.]
 [ 1.  1.  1.  0.]]
[[-3.  1.  3.  0.]
 [-3.  1.  3.  0.]
 [-3.  0.  3.  1.]
 [-2.  0.  2.  1.]]


In [11]:
# Agregar padding de tamaño 1
padded_matrix = add_padding(matrix, padding_size=1)
print(padded_matrix)


[[0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 1 1 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 1 1 1 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]


In [12]:
# Aplicar convolución horizontal con stride de 2
stride_conv = apply_stride_convolution(matrix, kernel_type='horizontal', stride=2)
print(stride_conv)


[[ 0.  0.]
 [-1. -1.]]


In [15]:

feature_maps = generate_feature_maps(matrix, n=4)
print(feature_maps.shape) 


(4, 4, 4)


In [14]:
# Aplicar max pooling con stride de 2
pooled_matrix = max_pooling(matrix, stride=2)
print(pooled_matrix)


[[1. 1. 0.]
 [0. 1. 0.]
 [1. 1. 0.]]
